In [2]:
import torch
import torchvision
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# 1. 전처리 (흑백 → RGB, 28x28 → 224x224)
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# 2. 데이터셋
train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

# 3. 사전 학습된 ResNet18 불러오기
model = models.resnet18(pretrained=True)

# 4. 마지막 fc 레이어 교체
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10)

# 5. 필요한 파라미터만 학습
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True

# 6. 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 7. 손실함수 & 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# 8. 학습 루프 + 정확도 측정
for epoch in range(5):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    # 평가 (테스트 데이터셋 정확도)
    model.eval()
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    train_acc = 100 * correct_train / total_train
    test_acc = 100 * correct_test / total_test
    avg_loss = running_loss / len(train_loader)

    print(f"Epoch {epoch+1} | Loss: {avg_loss:.4f} | Train Acc: {train_acc:.2f}% | Test Acc: {test_acc:.2f}%")


Epoch 1 | Loss: 0.6148 | Train Acc: 79.91% | Test Acc: 84.30%
Epoch 2 | Loss: 0.4361 | Train Acc: 84.59% | Test Acc: 85.18%
Epoch 3 | Loss: 0.4105 | Train Acc: 85.37% | Test Acc: 85.46%
Epoch 4 | Loss: 0.3946 | Train Acc: 85.82% | Test Acc: 85.01%
Epoch 5 | Loss: 0.3828 | Train Acc: 86.21% | Test Acc: 85.51%
